In [ ]:
#@title Spreadsheet Tab Names
E_form = "Fall Week 9 (11.20.22 - 11.26.22)" #@param {type:"string"}
D_form = "Fall 22 Week 9" #@param {type:"string"}

# Imports and Hidden Stuff

In [ ]:
import pickle
from apiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.colab import auth
import gspread
from google.auth import default

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import smtplib
from configparser import ConfigParser
from os.path import basename
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication

In [ ]:
def remove_end_spaces(string):
    string = "".join(string.rstrip())
    return "".join(string.rstrip().lstrip())

In [ ]:
# Dictionary:
# - Key: [First name] [Last name]
# - Value: Link to google calendar private ics link
# TODO: (High priority) Load from json file
ical = {}

In [ ]:
# Bad system to split shifts by, but this was caused becayse the way schedules 
# were organized changed drastically from when I started coding this to now.
# This was also my first large scale coding project outside of school, so 
# I didn't know the proper frameworks to make this scalable and robust.
# TODO: (Low priority) Reformat system to be intuitive and less redundant
days = ["SUNDAY", "MONDAY", "TUESDAY", "WEDNESDAY", "THURSDAY", "FRIDAY", "SATURDAY"]
hours = {
    "11:45PM": 0,
    "12:00AM": 0,
    "5:45AM": 6,
    "6:00AM": 6,
    "7:00AM": 7,
    "8:00AM": 8,
    "8:45AM": 9,
    "9:00AM": 9,
    "10:00AM": 10,
    "11:00AM": 11,
    "11:45AM": 12,
    "12:00PM": 12,
    "1:00PM": 13,
    "2:00PM": 14,
    "2:45PM": 15,
    "3:00PM": 15,
    "4:00PM": 16,
    "5:00PM": 17,
    "5:45PM": 18,
    "6:00PM": 18,
    "6:30PM": 18.5,
    "7:00PM": 19,
    "7:30PM": 19.5,
    "8:00PM": 20,
    "8:30PM": 20.5,
    "8:45PM": 21,
    "9:00PM": 21,
    "9:30PM": 21.5,
    "10:00PM": 22,
    "10:30PM": 22.5,
    "11:00PM": 23,
}

# Auth

In [ ]:
# For spreadsheet access
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

In [ ]:
# https://stackoverflow.com/questions/13694811/cannot-get-service-account-authorization-to-work-on-gcs-script-using-python-clie
# https://gist.github.com/nikhilkumarsingh/8a88be71243afe8d69390749d16c8322

# For google calendar access
scopes = ['https://www.googleapis.com/auth/calendar']
flow = InstalledAppFlow.from_client_secrets_file("/content/drive/MyDrive/Colab Notebooks/credentials.json", scopes=scopes)
credentials = flow.run_console()
pickle.dump(credentials, open("token.pkl", "wb"))
credentials = pickle.load(open("token.pkl", "rb"))
service = build("calendar", "v3", credentials=credentials)

# Deconstruct Spreadsheets

In [ ]:
# TODO: Parallelize
# TODO: Add exception/unit tests to ensure no bugs occurred

#defining my worksheet
E_worksheet = gc.open("Evergreen IQ Schedule").worksheet(E_form)
# Get_all_values gives a list of rows
rows = E_worksheet.get_all_values()
# Convert to a DataFrame 
E_df = pd.DataFrame(rows)

# Get dictionary of days
dates = (E_df.iloc[0,1:8]).to_numpy()
daydict = {}
for dayIX in range(len(days)):
    daydict[days[dayIX]] = dates[dayIX]

# Separate Schedule
E_schedule = E_df.iloc[:, 0:8]
E_schedule = E_schedule.iloc[[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]] # Due to odd formating, these are the only rows with schedule info

# Find worker information
E_finder = E_df.iloc[:, 9] 
E_start = E_finder[E_finder == "EVERGREEN IQ STAFF"].index.tolist()[0] + 2
E_end = E_finder[E_finder == "TOTAL HOURS"].index.tolist()[0]

# Zip worker information into an easier format
E_workers_info = E_df.iloc[E_start:E_end, 9:15] 
E_workers = np.array(list(zip(E_workers_info.iloc[:,1].values, E_workers_info.iloc[:,0].values, E_workers_info.iloc[:,3].values)))
print(E_workers)

In [ ]:
for workerIX in E_workers:
    workerIX[0] = remove_end_spaces(workerIX[0])
    workerIX[1] = remove_end_spaces(workerIX[1])

    name = (workerIX[0].capitalize() + " " + workerIX[1].capitalize())
    if not name in ical:
      print(name)

In [ ]:
# https://karenapp.io/articles/how-to-automate-google-calendar-with-python-using-the-calendar-api/
# https://github.com/kuzmoyev/google-calendar-simple-api
# https://cloud.google.com/iam/docs/service-accounts
# https://stackoverflow.com/questions/70495445/google-calendar-api-error-code-403-the-request-is-missing-a-valid-api-key
def evergreen_schedule(worker, schedule):
    shifts = []
    total_hours = 0
    worker_space = worker + " "
    for dayIX in range(1, 8):
        for timeIX in range(0, 14):
            timeIX = timeIX
            shift = schedule.iloc[timeIX, dayIX]
            shift = shift.splitlines()
            if shift:
                shift = remove_end_spaces(shift[-1])
                if shift == shift and (worker.lower() == shift.lower()
                                      or (worker_space.lower() in shift.lower()
                                          and ("(" in shift or "@" in shift))
                                      or (worker_space.lower() in shift.lower()
                                          and "ARHM" in shift)):
                    day = days[dayIX - 1]
                    time = schedule.iloc[timeIX, 0]
                    if time == "" or time == "\n" or time == " ":
                        time = schedule.iloc[timeIX-1, 0]
                    begin = time.split(" - ", 1)[0]
                    begin = begin[:-2] + ":00" + begin[-2:]
                    end = time.split(" - ", 1)[1]
                    end = end[:-2] + ":00" + end[-2:]

                    reason = ""
                    if worker.lower() != shift.lower():
                        temp = shift.capitalize() # Messes up if one part isn't capitalized
                        reason = temp.split(worker, 1)[1]
                        reason = "    |###| Please Note:" + reason

                    shifts.append([day, [begin, end], " @ Evergreen", hours[begin], reason])
                    if hours[end] < hours[begin]:
                        total_hours += 24 + hours[end] - hours[begin]
                    else:
                        total_hours += hours[end] - hours[begin]

    for dayIX in days:
        shiftIX = 0
        while shiftIX < (len(shifts) - 1):
            if dayIX.lower() == shifts[shiftIX][0].lower() and dayIX.lower() == shifts[shiftIX + 1][0].lower():
                if shifts[shiftIX][1][1] == shifts[shiftIX + 1][1][0]:
                    shifts[shiftIX][1][1] = shifts[shiftIX + 1][1][1]
                    del shifts[shiftIX + 1]
                    shiftIX -= 1
            shiftIX += 1

    # Reformat to strings
    for shiftIX in shifts:
        begin = shiftIX[1][0]
        end = shiftIX[1][1]
        shiftIX[1] = begin + "-" + end
        start_time = datetime.strptime(daydict[shiftIX[0]] + " " + shiftIX[1].split('-')[0], '%m/%d/%y %I:%M%p')
        end_time = datetime.strptime(daydict[shiftIX[0]] + " " + shiftIX[1].split('-')[1], '%m/%d/%y %I:%M%p')

        if end_time < start_time:
            end_time += timedelta(days=1)
        

        shiftIX.append(start_time)
        shiftIX.append(end_time)

    return shifts, total_hours

In [ ]:
D_worksheet = gc.open(D_form).get_worksheet(0)
#get_all_values gives a list of rows
rows = D_worksheet.get_all_values()
# Convert to a DataFrame 
D_df = pd.DataFrame(rows)

# Separate Schedule
D_schedule = D_df.iloc[2:16, 0:8]
DS_column = D_df.iloc[:, 0]
DS_end = DS_column[DS_column == "SUPERVISORS"].index.tolist()[0]
DS_schedule = D_df.iloc[17:DS_end, 0:8]

# Get week
week = D_df.iat[0,0]

# Find worker information
D_finder = D_df.iloc[:, 11] 
D_end = D_finder[D_finder == "TOTAL HOURS"].index.tolist()[0]

# Zip worker information into an easier format
D_workers_info = D_df.iloc[1:D_end, 11:16] 
D_workers = np.array(list(zip(D_workers_info.iloc[:,3].values, D_workers_info.iloc[:,2].values, D_workers_info.iloc[:,1].values)))
D_workers = D_workers[6:] # Sorry, no supervisors allowed
print(D_workers)

In [ ]:
for workerIX in D_workers:
    workerIX[0] = remove_end_spaces(workerIX[0])
    workerIX[1] = remove_end_spaces(workerIX[1])

    name = (workerIX[0].capitalize() + " " + workerIX[1].capitalize())
    if not name in ical:
      print(name)

In [ ]:
def special_schedule(schedule, worker):
    shifts = []
    total_hours = 0
    worker_space = worker + " "

    for dayIX in range(1, 8):
        for timeIX in range(len(schedule.index)):
            shift = schedule.iloc[timeIX, dayIX]
            if shift == shift and (worker.lower() == shift.lower()
                                   or (worker_space.lower() in shift.lower()
                                       and ("(" in shift or "@" in shift))):
                shift = remove_end_spaces(shift)
                day = days[dayIX - 1]
                # Find Time
                found = False
                IX = 1
                time = ""
                while found == False:
                    potential = schedule.iloc[timeIX - IX, dayIX]
                    print(potential)
                    if ("AM".lower() in potential.lower() or "PM".lower() in potential.lower()) and "-" in potential:
                        time = potential
                        found = True
                    IX += 1

                bad_flag = False

                begin = ""
                end = ""
                if " - " in time:
                    begin = time.split(" - ", 1)[0]
                    end = time.split(" - ", 1)[1]
                    if begin.count(":") == 0:
                        begin = begin[:-2] + ":00" + begin[-2:].upper()
                        end = end[:-2] + ":00" + end[-2:].upper()
                elif "-" in time:
                    begin = time.split("-", 1)[0]
                    end = time.split("-", 1)[1]
                    if begin.count(":") == 0:
                        begin = begin[:-2] + ":00" + begin[-2:].upper()
                        end = end[:-2] + ":00" + end[-2:].upper()
                else:
                    bad_flag = True

                if not bad_flag:
                    reason = ""
                    if worker.lower() != shift.lower():
                        reason = shift.split(worker, 1)[1]
                        reason = "    |###| Please Note:" + reason

                    time = begin + "-" + end

                    start_time = datetime.strptime(daydict[day] + " " + begin, '%m/%d/%y %I:%M%p')
                    end_time = datetime.strptime(daydict[day] + " " + end, '%m/%d/%y %I:%M%p')

                    if end_time < start_time:
                        end_time += timedelta(days=1)

                    shifts.append([day, time, " @ Special DDFD Shift", hours[begin], reason, start_time, end_time])
                    if hours[end] < hours[begin]:
                        total_hours += 24 + hours[end] - hours[begin]
                    else:
                        total_hours += hours[end] - hours[begin]

    return shifts, total_hours

def DDFD_schedule(worker, schedule, DS_schedule):
    worker_space = worker + " "
    shifts = []
    total_hours = 0

    for dayIX in range(1, 8):
            for timeIX in range(len(schedule.index)):
                shift = schedule.iloc[timeIX, dayIX]
                if shift == shift and (worker.lower() == shift.lower()
                                      or (worker_space.lower() in shift.lower()
                                          and ("(" in shift or "@" in shift))):
                    shift = remove_end_spaces(shift)
                    day = days[dayIX - 1]
                    begin = None
                    end = None
                    if "BUCKET" in schedule.iloc[timeIX, 0]:
                        time = "6:00PM-9:00PM"
                        begin = "6:00PM"
                        end = "9:00PM"
                    else:
                        time = schedule.iloc[timeIX, 0]
                        begin = time.split("-", 1)[0]
                        end = time.split("-", 1)[1]

                    start_time = datetime.strptime(daydict[day] + " " + begin, '%m/%d/%y %I:%M%p')
                    end_time = datetime.strptime(daydict[day] + " " + end, '%m/%d/%y %I:%M%p')

                    if begin == "11:45PM":
                        start_time -= timedelta(days=1)
                    if begin == "8:45PM":
                        end_time += timedelta(days=1)

                    reason = ""
                    if worker.lower() != shift.lower():
                        reason = shift.split(worker, 1)[1]
                        reason = "    |###| Please Note:" + reason

                    shifts.append([day, time, " @ DDFD", hours[begin], reason, start_time, end_time])
                    if hours[end] < hours[begin]:
                        total_hours += 24 + hours[end] - hours[begin]
                    else:
                        total_hours += hours[end] - hours[begin]

    # Special Shifts
    sshifts, shours = special_schedule(DS_schedule, worker)
    shifts += sshifts

    # Sort schedule to be chronological
    shifts.sort(key=lambda x: x[3], reverse=False)

    return shifts, total_hours, shours

# Add to Calendar and Send Email

In [ ]:
def reset_calendar_week(service, min, max):
    result = service.calendarList().list().execute()
    for resultIX in result["items"]:
        calendar_id = resultIX['id']
        event_result = service.events().list(calendarId=calendar_id, timeMin=min, timeMax=max, maxResults=9999).execute()
        for eventIX in event_result["items"]:
            service.events().delete(calendarId=calendar_id, eventId=eventIX["id"]).execute()

In [ ]:
def add_to_calendar(shifts, full_name, service):
    for shiftIX in shifts:
        location = None
        title = None
        calendar = full_name
        if "Evergreen" in shiftIX[2]:
            location = "Evergreen Front Desk"
            title = "Evergreen Front Desk Shift"
        elif "Special" in shiftIX[2]:
            location = "De Neve Front Desk"
            title = "De Neve Front Desk Special Shift"
        elif "DDFD" in shiftIX[2]:
            location = "De Neve Front Desk"
            title = "De Neve Front Desk Shift"

        event = {
          'summary': title,
          'location': location,
          'description': 'Front Desk Shift. Additional Notes: ' + shiftIX[4],
          'start': {
            'dateTime': shiftIX[5].strftime("%Y-%m-%dT%H:%M:%S"),
            'timeZone': "America/Los_Angeles",
          },
          'end': {
            'dateTime': shiftIX[6].strftime("%Y-%m-%dT%H:%M:%S"),
            'timeZone': "America/Los_Angeles",
          },
          'reminders': {
            'useDefault': False,
            'overrides': [
              {'method': 'email', 'minutes': 3 * 60},
              {'method': 'popup', 'minutes': 15},
            ],
          },
        }
        # print(shiftIX[5].strftime("%Y-%m-%dT%H:%M:%S"), shiftIX[6].strftime("%Y-%m-%dT%H:%M:%S"))
        for calendar_list_entry in calendar_result['items']:
            if calendar in calendar_list_entry["summary"]:
                service.events().insert(calendarId=calendar_list_entry["id"], body=event).execute()

In [ ]:
def make_message(name, l_name, shifts, total_hours, shours):
    message = f"""Good afternoon {name},

I am a bot created by Blake Gella. Here is your tenative schedule for next week.\n"""
    message += "\n"
    message += str(week + "\n")
    message += "Total Main Hours: " + str(total_hours) + "\n"
    if shours != 0:
        message += "Special Shift Hours: " + str(shours) + "    |###| Please note this may be less\n"
    else:
        message += "Special Shift Hours: " + str(shours) + "\n"
    message += "\n"

    counter = 0
    for dayIX in days:
        message += dayIX + " " + dates[counter] + ":\n"

        working = False
        for shiftIX in shifts:
            if dayIX.lower() == shiftIX[0].lower():
                message += shiftIX[1] + shiftIX[2] + shiftIX[4] + "\n"
                working = True
        if not working:
            message += "NO WORK :^)\n"

        message += "\n"
        counter += 1

    message += """
Here's also a personalized ical link that will subscribe you to a calendar:

"""
    message += ical[name + " " + l_name]

    message += """

That's all! Please check the schedule still in case I made any errors. Have a great day!

Sincerely,
Glitch Bot

P.S.
The link I have included is for a .ical file that you can use to subscribe to a work calendar! If you are on an iPhone, please open this email in the Apple mail app to import the events.
------------------------------------------------
Beep-Boop. I am a bot created by Blake Gella. If you notice any errors, please contact Blake ASAP!"""
    return message

# Main

In [ ]:
min = (datetime.strptime(daydict["SUNDAY"] + " 00:01", '%m/%d/%y %H:%M')+ timedelta(hours=8)).strftime("%Y-%m-%dT%H:%M:%S")
max = (datetime.strptime(daydict["SATURDAY"] + " 00:00", '%m/%d/%y %H:%M')+ timedelta(days=1, hours=8)).strftime("%Y-%m-%dT%H:%M:%S")
reset_calendar_week(service, min+"Z", max+"Z")

config_object = ConfigParser()
config_object.read("/content/drive/MyDrive/Colab Notebooks/config.ini")
password = config_object["bot_info"]["password"]
bot_email = #REPLACE
host_email =#REPLACE
port = 465
s = smtplib.SMTP_SSL("smtp.gmail.com", port)
s.login(bot_email, password)

calendar_result = service.calendarList().list().execute()
print("Evergreen")
for workerIX in E_workers:
    workerIX[0] = remove_end_spaces(workerIX[0])
    workerIX[1] = remove_end_spaces(workerIX[1])

    print("Fetching Evergreen Schedules")
    E_shifts, E_total_hours = evergreen_schedule(workerIX[0], E_schedule)
    shifts = E_shifts
    total_hours = E_total_hours

    if total_hours != 0:
        add_to_calendar(shifts, workerIX[0] + " " + workerIX[1], service)

        print("Creating Message")
        message = make_message(workerIX[0].capitalize(), workerIX[1].capitalize(), shifts, total_hours, 0)
        print("Message Created. Creating Email")

        msg = MIMEMultipart()
        msg['Subject'] = f"Updated Schedule Delivery for {week}!"
        msg['From'] = bot_email
        msg['To'] = host_email
        msg.attach(MIMEText(message, 'plain'))
        s.sendmail("", workerIX[2], msg.as_string()) bot email
        print("Email Sent!")
    print(workerIX[0] + ": " + str(total_hours))
    print()

print("DDFD")

for workerIX in D_workers:
    workerIX[0] = remove_end_spaces(workerIX[0])
    workerIX[1] = remove_end_spaces(workerIX[1])

    print("Fetching DDFD Schedules")
    D_shifts, D_total_hours, S_hours = DDFD_schedule(workerIX[0], D_schedule, DS_schedule)
    shifts = D_shifts
    total_hours = D_total_hours

    if total_hours != 0 or S_hours != 0:
        add_to_calendar(shifts, workerIX[0] + " " + workerIX[1], service)

        print("Creating Message")
        message = make_message(workerIX[0].capitalize(), workerIX[1].capitalize(), shifts, total_hours, S_hours)
        print("Message Created. Creating Email")

        msg = MIMEMultipart()
        msg['Subject'] = f"Updated Schedule Delivery for {week}!"
        msg['From'] = bot_email
        msg['To'] = host_email
        msg.attach(MIMEText(message, 'plain'))
        s.sendmail(bot_email, workerIX[2], msg.as_string())
        print("Email Sent!")
    print(workerIX[0] + ": " + str(total_hours) + "/" + str(S_hours))
    print()

# Troubleshooting

In [ ]:
min = (datetime.strptime(daydict["SUNDAY"] + " 00:01", '%m/%d/%y %H:%M')+ timedelta(hours=8)).strftime("%Y-%m-%dT%H:%M:%S")
max = (datetime.strptime(daydict["SATURDAY"] + " 00:00", '%m/%d/%y %H:%M')+ timedelta(days=1, hours=8)).strftime("%Y-%m-%dT%H:%M:%S")
reset_calendar_week(service, min+"Z", max+"Z")

calendar_result = service.calendarList().list().execute()
print("Evergreen")
for workerIX in E_workers:
    workerIX[0] = remove_end_spaces(workerIX[0])
    workerIX[1] = remove_end_spaces(workerIX[1])

    print("Fetching Evergreen Schedules")
    E_shifts, E_total_hours = evergreen_schedule(workerIX[0], E_schedule)
    shifts = E_shifts
    total_hours = E_total_hours

    if total_hours != 0:
        add_to_calendar(shifts, workerIX[0] + " " + workerIX[1], service)

print("DDFD")
for workerIX in D_workers:
    workerIX[0] = remove_end_spaces(workerIX[0])
    workerIX[1] = remove_end_spaces(workerIX[1])

    print("Fetching DDFD Schedules")
    D_shifts, D_total_hours, S_hours = DDFD_schedule(workerIX[0], D_schedule, DS_schedule)
    shifts = D_shifts
    total_hours = D_total_hours

    if total_hours != 0 or S_hours != 0:
        add_to_calendar(shifts, workerIX[0] + " " + workerIX[1], service)